In [7]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [8]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles
pose = mp_pose.Pose()


In [9]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [10]:
def extract_keypoints(results):
    lw = np.array([[res.x, res.y, res.z, res.visibility] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST.value]]]).flatten() if results.pose_landmarks else np.zeros(33*4)
    rw = np.array([[res.x, res.y, res.z, res.visibility] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value]]]).flatten() if results.pose_landmarks else np.zeros(33*4)
    pose = pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    result = np.zeros(132)
    result[15]=pose[15]
    result[16]=pose[16]
    result = result.flatten()

    return np.concatenate([result])
    
def extract_keypoints1(results):
    lw = np.array([[res.x, res.y, res.z, res.visibility] for res in [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST.value]]]).flatten() if results.pose_landmarks else np.zeros(4)
    return np.concatenate([lw])

def extract_keypoints2(landmarks):
  wrist_keypoints = []
  # Loop through the landmarks in the pose landmark object
  for landmark in landmarks:
    # Check if the landmark is a wrist keypoint
    if landmark.type == 'WRIST':
      # Extract the x, y, z, and visibility values for the wrist keypoint
      x, y, z, visibility = landmark.x, landmark.y, landmark.z, landmark.visibility
      wrist_keypoints.append([x, y, z, visibility])
  return np.array(wrist_keypoints)

In [11]:
DATA_PATH = os.path.join('./Datasets') 

# Actions that we try to detect
actions = np.array(['Normal'])

# Thirty videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 60

# Folder start


In [13]:

# Set mediapipe model 
start_folder = 1
number = 0
VIDEO_PATH = "./assets/videos/"+str(number)+".mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
count = 0
start = False
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        success, frame = cap.read()
        if not success:
            break
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        image = cv2.resize(image,(700,700))
        #For landmarks drawing
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        if start:
            keypoints = extract_keypoints2(results.pose_landmarks.landmark)
            npy_path = os.path.join(DATA_PATH, 'try', str(number), str(count))
            np.save(npy_path, keypoints)
            count+=1
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('s'):
            start = True
            print("Starting to collect for folder "+ str(number))
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

Starting to collect for folder 0


AttributeError: type

In [3]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
DATA_PATH="./Datasets/"
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [7]:
X = np.array(sequences)
print(X.shape)

(10, 60, 4)


In [8]:
y = to_categorical(labels).astype(int)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout,GRU
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(units=8, input_shape=(60, 4), kernel_regularizer='l2', recurrent_regularizer='l2', bias_regularizer='l2', return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(actions.shape[0], activation='sigmoid'))

In [19]:
model.compile(optimizer=Adam(0.002), loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=30,batch_size=32, callbacks=[tb_callback],validation_data=(X_test,y_test))
# model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 0.9216 - accuracy: 0.2167 - val_loss: 0.8872 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 55ms/step - loss: 0.8997 - accuracy: 0.3125 - val_loss: 0.8723 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 61ms/step - loss: 0.9007 - accuracy: 0.2896 - val_loss: 0.8575 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 71ms/step - loss: 0.8765 - accuracy: 0.3167 - val_loss: 0.8429 - val_accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 58ms/step - loss: 0.8457 - accuracy: 0.3813 - val_loss: 0.8284 - val_accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 63ms/step - loss: 0.8347 - accuracy: 0.3771 - val_loss: 0.8141 - val_accuracy: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 60ms/step - loss: 0.8191 - accuracy: 0.3771 - val_loss: 0.7999 - val_accuracy: 0.0000e+00
E

In [ ]:
res = model.predict(X_test,batch_size=30)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]